# Prepare Dataset For YOLOv7 

* `X1`,`Y1`,`X2`,`Y2` -> **Data format**

* `class_label`,`x_center`,`y_center`,`width`,`height` -> **target format**

In [50]:
import os
from math import sqrt

In [51]:
def clear_file_Content(fileContent):
    content_list = []
    # Clear the file content
    for file in fileContent:
        content_list.append(file.strip('\n'))
        
    return content_list

In [52]:
from PIL import Image
import os.path

In [53]:
def modify_folder_data(path,path_image, folder_path):
    for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
        filepath = os.path.join(path, filename)
        with open(filepath, mode='r+') as f:
            fileContent = f.readlines()
            cleaned_file_content = clear_file_Content(fileContent)
            new_lines = []
            #img_name = filename.split('.')[0]
            #img = Image.open(f"{path_image}/{img_name}.tif")
            #print(img.size)
            for line in cleaned_file_content:
                xs = line.split(',')
                xs = [float(i) for i in xs]
                # xs[0]= x1 xs[1]= y1 xs[2]= x2 xs[3]= y2
        
                xmin = xs[0]
                ymin = xs[1]
                xmax = xs[2]
                ymax = xs[3]
                
                width = abs(xmax-xmin)
                height = abs(ymax-ymin)
                x_center = (xmin+xmax) / 2
                y_center = (ymin+ymax) / 2

                width /= 1000 
                height /= 1000
                x_center /= 1000
                y_center /= 1000
                
                #print(f"width: {width} height: {height} x_center: {x_center} y_center: {y_center}")
                
                new_format = [0,x_center,y_center,width,height]
                new_format_string = ' '.join(str(item) for item in new_format)
                new_lines.append(new_format_string)
            with open(f"{folder_path}/{filename}", "w") as created_file:
                for line in new_lines:
                    created_file.write("%s\n" % line)
                

In [54]:
path_TrainGroundTruth = "SignatureObjectDetection\TrainGroundTruth"
path_TestGroundTruth = "SignatureObjectDetection\TestGroundTruth"
path_image_train = "SignatureObjectDetection\TrainImages"
path_image_test = "SignatureObjectDetection\TestImages"

In [55]:
modify_folder_data(path_TrainGroundTruth,path_image_train, "SignatureObjectDetection/Data/Train/labels/")
modify_folder_data(path_TestGroundTruth,path_image_test, "SignatureObjectDetection/Data/Test/labels/")

# Resize All Images

In [56]:
from PIL import Image                                              
import os, sys                       

path = "SignatureObjectDetection\\TestImages\\"
dirs = os.listdir( path )                                       

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            #print(item)
            im = Image.open(path+item)
            #f, e = os.path.splitext(path+item)
            imResize = im.resize((1024,1024), Image.ANTIALIAS)
            imResize.save(f"SignatureObjectDetection/Data/Test/images/{item}")

resize()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_20388\2820256537.py:13: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((1024,1024), Image.ANTIALIAS)


In [57]:
from PIL import Image                                              
import os, sys                       

path = "SignatureObjectDetection\\TrainImages\\"
dirs = os.listdir( path )                                       

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            #f, e = os.path.splitext(path+item)
            imResize = im.resize((1024,1024), Image.ANTIALIAS)
            imResize.save(f"SignatureObjectDetection/Data/Train/images/{item}")

resize()

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_20388\3453852455.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((1024,1024), Image.ANTIALIAS)


# Split Data into Train Validation sets

In [58]:
path = "SignatureObjectDetection\Data\Train\labels\\"
all_names = []
for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
    all_names.append(filename.split('.')[0])

In [59]:
req = len(all_names)//5
req

132

In [60]:
import random

In [61]:
selected_names = random.sample(all_names,req )
selected_names[:10]

['197', '21', '373', '171', '97', '581', '158', '184', '129', '586']

In [62]:
import shutil

In [63]:
def train_validate_split_groundTruth(path):
    for filename in filter(lambda p: p.endswith("txt"), os.listdir(path)):
        if selected_names.__contains__(filename.split('.')[0]):
            filepath = os.path.join(path, filename) 
            shutil.move(filepath, f"SignatureObjectDetection/Data/Validation/labels/{filename}")
        
        
def train_validate_split_images(path):
    for filename in filter(lambda p: p.endswith("tif"), os.listdir(path)):
        if selected_names.__contains__(filename.split('.')[0]):
            filepath = os.path.join(path, filename) 
            shutil.move(filepath, f"SignatureObjectDetection/Data/Validation/images/{filename}")
        
        
train_validate_split_groundTruth("SignatureObjectDetection\\Data\\Train\\labels")     
train_validate_split_images("SignatureObjectDetection\\Data\\Train\\images")